# BoostDM-CH performance with list of artifacts


### List of artifacts from Vlasschaert C, Mack T, Heimlich JB, Niroula A, Uddin MM, Weinstock J, et al. A practical approach to curate clonal hematopoiesis of indeterminate potential in human genetic data sets. Blood 2023;141: 2214–23.

In [21]:
import pandas as pd
import os
from io import StringIO
import copy
import gzip
import matplotlib.pyplot as plt

# 1. Prepare matrix

In [22]:
### Annotate with boostDM-CH prediction
Rules_matrix = pd.read_csv('../../Paper_data/Expert_curated_rules/genes12_BoostdmCH_Harvard_run20230803.tsv.gz', sep='\t')
# Include an id per mutation
Rules_matrix['ID2'] = Rules_matrix['chr'].astype(str)+'-'+Rules_matrix['pos'].astype(str)+'-'+ Rules_matrix['alt'].astype(str)
Rules_matrix

,gene,ENSEMBL_TRANSCRIPT,ENSEMBL_GENE,chr,pos,alt,aachange,CLUSTL_SCORE,CLUSTL_cat_1,HotMaps_cat_1,...,shap_csqn_type_nonsense,shap_csqn_type_splicing,shap_csqn_type_synonymous,row,Prot_pos,Niroula,Bick,CNIC,WHO,ID2
0,ASXL1,ENST00000375687,ENSG00000171456,20,32358779,C,K2Q,0.0,0,0,...,-2.783079,-0.099647,0.030009,0,2,0.0,0.0,0.0,0.0,20-32358779-C
1,ASXL1,ENST00000375687,ENSG00000171456,20,32358779,G,K2E,0.0,0,0,...,-2.783079,-0.099647,0.030009,1,2,0.0,0.0,0.0,0.0,20-32358779-G
2,ASXL1,ENST00000375687,ENSG00000171456,20,32358779,T,K2*,0.0,0,0,...,2.419656,-0.038151,0.070408,2,2,0.0,0.0,0.0,0.0,20-32358779-T
3,ASXL1,ENST00000375687,ENSG00000171456,20,32358780,C,K2T,0.0,0,0,...,-2.774794,-0.099765,0.029898,3,2,0.0,0.0,0.0,0.0,20-32358780-C
4,ASXL1,ENST00000375687,ENSG00000171456,20,32358780,G,K2R,0.0,0,0,...,-2.774794,-0.099765,0.029898,4,2,0.0,0.0,0.0,0.0,20-32358780-G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87934,U2AF1,ENST00000291552,ENSG00000160201,21,43107490,C,A2G,0.0,0,0,...,0.000000,0.000000,0.000005,2146,2,0.0,0.0,0.0,0.0,21-43107490-C
87935,U2AF1,ENST00000291552,ENSG00000160201,21,43107490,T,A2E,0.0,0,0,...,0.000000,0.000000,0.000005,2147,2,0.0,0.0,0.0,0.0,21-43107490-T
87936,U2AF1,ENST00000291552,ENSG00000160201,21,43107491,A,A2S,0.0,0,0,...,0.000000,0.000000,0.000005,2148,2,0.0,0.0,0.0,0.0,21-43107491-A
87937,U2AF1,ENST00000291552,ENSG00000160201,21,43107491,G,A2P,0.0,0,0,...,0.000000,0.000000,0.000005,2149,2,0.0,0.0,0.0,0.0,21-43107491-G


In [23]:
# List of 52 recurrent SNVs identified as drivers by the Bick rule that are false positives

Artifacts = pd.read_csv('../../Paper_data/Expert_curated_rules/Artifacts_Bick_et_al.csv', sep='\t')
Artifacts['AA_mut'] = Artifacts['AA_mut'].str.replace('p.', '').str.replace('X', '*')
Artifacts

/tmp/jobs/jramis/9874679/ipykernel_28750/1363310182.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  Artifacts['AA_mut'] = Artifacts['AA_mut'].str.replace('p.', '').str.replace('X', '*')


,Cohort,Gene,Variants,Transcript,exon,cDNA_mut,AA_mut
0,Germline,DNMT3A,DNMT3A,NM_022552,exon8,c.G892C,G298R
1,Germline,TET2,TET2,NM_001127208,exon10,c.C4318T,R1440W
2,Germline,TET2,TET2,NM_001127208,exon11,c.G5978A,R1993Q
3,Germline,TET2,TET2,NM_001127208,exon10,c.A4334G,Q1445R
4,Germline,TET2,TET2,NM_001127208,exon3,c.C3365A,T1122K
5,Germline,TET2,TET2,NM_001127208,exon3,c.C3326T,S1109F
6,Germline,TET2,TET2,NM_001127208,exon10,c.G4276A,V1426M
7,Germline,TET2,TET2,NM_001127208,exon11,c.C5879T,S1960L
8,Germline,TET2,TET2,NM_001127208,exon6,c.C3737T,S1246L
9,Germline,TET2,TET2,NM_001127208,exon10,c.G4319A,R1440Q


In [24]:
# Get reference and altered nt depending on the direction of the gene (negative or positive strand)

def dna_rev(nt):
    if nt == 'A':
        return 'T'
    elif nt == 'T':
        return 'A'
    elif nt == 'C':
        return 'G'
    elif nt == 'G':
        return 'C'

ref = []
alt = []
for x in range(len(Artifacts)):
    row = Artifacts.iloc[x,:]
    if row.Gene in ['TET2', 'ASXL1', 'PPM1D']:
        ref.append(row.cDNA_mut[2])
        alt.append(row.cDNA_mut[-1])
    elif row.Gene in ['DNMT3A', 'TP53', 'U2AF1']:
        ref.append(dna_rev(row.cDNA_mut[2]))
        alt.append(dna_rev(row.cDNA_mut[-1]))
Artifacts['ref'] = ref
Artifacts['alt'] = alt
Artifacts

,Cohort,Gene,Variants,Transcript,exon,cDNA_mut,AA_mut,ref,alt
0,Germline,DNMT3A,DNMT3A,NM_022552,exon8,c.G892C,G298R,C,G
1,Germline,TET2,TET2,NM_001127208,exon10,c.C4318T,R1440W,C,T
2,Germline,TET2,TET2,NM_001127208,exon11,c.G5978A,R1993Q,G,A
3,Germline,TET2,TET2,NM_001127208,exon10,c.A4334G,Q1445R,A,G
4,Germline,TET2,TET2,NM_001127208,exon3,c.C3365A,T1122K,C,A
5,Germline,TET2,TET2,NM_001127208,exon3,c.C3326T,S1109F,C,T
6,Germline,TET2,TET2,NM_001127208,exon10,c.G4276A,V1426M,G,A
7,Germline,TET2,TET2,NM_001127208,exon11,c.C5879T,S1960L,C,T
8,Germline,TET2,TET2,NM_001127208,exon6,c.C3737T,S1246L,C,T
9,Germline,TET2,TET2,NM_001127208,exon10,c.G4319A,R1440Q,G,A


In [25]:
bdmchclasses = []
bdmchscores = []
for i in range(len(Artifacts)):
    row = Artifacts.iloc[i,:]
    aa = row.AA_mut
    row_gene = row.Gene
    row_alt = row.alt
    bdmchscore = list(set(Rules_matrix[(Rules_matrix['aachange']==aa)&(Rules_matrix['gene']==row_gene)&\
                                      (Rules_matrix['alt']==row_alt)]['boostDM_score']))
    bdmchscores.append(bdmchscore[0])
    bdmchclass = list(set(Rules_matrix[(Rules_matrix['aachange']==aa)&(Rules_matrix['gene']==row_gene)&\
                                      (Rules_matrix['alt']==row_alt)]['boostDM_class']))
    bdmchclasses.append(bdmchclass)
Artifacts['boostDM_score'] = bdmchscores
Artifacts['boostDM_class'] = [1 if x > 0.5 else 0 for x in Artifacts['boostDM_score']]
Artifacts

,Cohort,Gene,Variants,Transcript,exon,cDNA_mut,AA_mut,ref,alt,boostDM_score,boostDM_class
0,Germline,DNMT3A,DNMT3A,NM_022552,exon8,c.G892C,G298R,C,G,0.959155,1
1,Germline,TET2,TET2,NM_001127208,exon10,c.C4318T,R1440W,C,T,0.037335,0
2,Germline,TET2,TET2,NM_001127208,exon11,c.G5978A,R1993Q,G,A,0.005630,0
3,Germline,TET2,TET2,NM_001127208,exon10,c.A4334G,Q1445R,A,G,0.941834,1
4,Germline,TET2,TET2,NM_001127208,exon3,c.C3365A,T1122K,C,A,0.070532,0
5,Germline,TET2,TET2,NM_001127208,exon3,c.C3326T,S1109F,C,T,0.004361,0
6,Germline,TET2,TET2,NM_001127208,exon10,c.G4276A,V1426M,G,A,0.045444,0
7,Germline,TET2,TET2,NM_001127208,exon11,c.C5879T,S1960L,C,T,0.002747,0
8,Germline,TET2,TET2,NM_001127208,exon6,c.C3737T,S1246L,C,T,0.001692,0
9,Germline,TET2,TET2,NM_001127208,exon10,c.G4319A,R1440Q,G,A,0.073961,0


In [26]:
Artifacts = Artifacts[Artifacts['Cohort']!='All of Us'].reset_index(drop=True)
Artifacts

,Cohort,Gene,Variants,Transcript,exon,cDNA_mut,AA_mut,ref,alt,boostDM_score,boostDM_class
0,Germline,DNMT3A,DNMT3A,NM_022552,exon8,c.G892C,G298R,C,G,0.959155,1
1,Germline,TET2,TET2,NM_001127208,exon10,c.C4318T,R1440W,C,T,0.037335,0
2,Germline,TET2,TET2,NM_001127208,exon11,c.G5978A,R1993Q,G,A,0.005630,0
3,Germline,TET2,TET2,NM_001127208,exon10,c.A4334G,Q1445R,A,G,0.941834,1
4,Germline,TET2,TET2,NM_001127208,exon3,c.C3365A,T1122K,C,A,0.070532,0
5,Germline,TET2,TET2,NM_001127208,exon3,c.C3326T,S1109F,C,T,0.004361,0
6,Germline,TET2,TET2,NM_001127208,exon10,c.G4276A,V1426M,G,A,0.045444,0
7,Germline,TET2,TET2,NM_001127208,exon11,c.C5879T,S1960L,C,T,0.002747,0
8,Germline,TET2,TET2,NM_001127208,exon6,c.C3737T,S1246L,C,T,0.001692,0
9,Germline,TET2,TET2,NM_001127208,exon10,c.G4319A,R1440Q,G,A,0.073961,0


In [27]:
Artifacts['Cohort'] = Artifacts['Cohort'].apply(lambda x: 'Germline artifact' if x == 'Germline' else 'Genomic artifact')
Artifacts

,Cohort,Gene,Variants,Transcript,exon,cDNA_mut,AA_mut,ref,alt,boostDM_score,boostDM_class
0,Germline artifact,DNMT3A,DNMT3A,NM_022552,exon8,c.G892C,G298R,C,G,0.959155,1
1,Germline artifact,TET2,TET2,NM_001127208,exon10,c.C4318T,R1440W,C,T,0.037335,0
2,Germline artifact,TET2,TET2,NM_001127208,exon11,c.G5978A,R1993Q,G,A,0.005630,0
3,Germline artifact,TET2,TET2,NM_001127208,exon10,c.A4334G,Q1445R,A,G,0.941834,1
4,Germline artifact,TET2,TET2,NM_001127208,exon3,c.C3365A,T1122K,C,A,0.070532,0
5,Germline artifact,TET2,TET2,NM_001127208,exon3,c.C3326T,S1109F,C,T,0.004361,0
6,Germline artifact,TET2,TET2,NM_001127208,exon10,c.G4276A,V1426M,G,A,0.045444,0
7,Germline artifact,TET2,TET2,NM_001127208,exon11,c.C5879T,S1960L,C,T,0.002747,0
8,Germline artifact,TET2,TET2,NM_001127208,exon6,c.C3737T,S1246L,C,T,0.001692,0
9,Germline artifact,TET2,TET2,NM_001127208,exon10,c.G4319A,R1440Q,G,A,0.073961,0


In [31]:
# BoostDM-CH classification of the false positive SNV 
Artifacts['boostDM_class'].value_counts()

0    28
1    24
Name: boostDM_class, dtype: int64

In [35]:
# Num of false positive with BoostDM-CH score under 0.9
len(Artifacts[Artifacts['boostDM_score']<0.9])

34

In [33]:
# Genes with false positive SNV
Artifacts['Gene'].value_counts()

TET2      35
ASXL1     10
TP53       4
DNMT3A     1
PPM1D      1
U2AF1      1
Name: Gene, dtype: int64

In [34]:
# Gene specific
Artifacts_false_positives = Artifacts[Artifacts['Gene']=='TET2']
# Artifacts_false_positives = Artifacts_false_positives[Artifacts_false_positives['boostDM_class']==0]
len(Artifacts_false_positives[Artifacts_false_positives['boostDM_score']<0.9])

31